In [1]:
def negate(literal):
    pred, sign, args = literal
    return (pred, not sign, args)

def apply_subst_literal(literal, subst):
    pred, sign, args = literal
    return (pred, sign, [subst.get(a, a) for a in args])

def unify(x, y, subst):
    if subst is None:
        return None
    if x == y:
        return subst
    if isinstance(x, str) and x.islower():  # variable
        return unify_var(x, y, subst)
    if isinstance(y, str) and y.islower():
        return unify_var(y, x, subst)
    if isinstance(x, list) and isinstance(y, list) and len(x) == len(y):
        for a, b in zip(x, y):
            subst = unify(a, b, subst)
        return subst
    return None

def unify_var(var, x, subst):
    if var in subst:
        return unify(subst[var], x, subst)
    if x in subst:
        return unify(var, subst[x], subst)
    new = subst.copy()
    new[var] = x
    return new

def resolve(c1, c2):
    resolvents = []
    for l1 in c1:
        for l2 in c2:
            if l1[0] == l2[0] and l1[1] != l2[1]:
                subst = unify(l1[2], l2[2], {})
                if subst is not None:
                    new_clause = (
                        [apply_subst_literal(x, subst) for x in c1 if x != l1] +
                        [apply_subst_literal(x, subst) for x in c2 if x != l2]
                    )
                    # remove duplicates
                    cleaned = []
                    for c in new_clause:
                        if c not in cleaned:
                            cleaned.append(c)
                    resolvents.append(cleaned)
    return resolvents


# ------------ KB from PPT (in clause form) ------------

KB = [
    # a. John likes all food:  Food(x) → Likes(John,x)
    [( "Food", False, ["x"] ), ("Likes", True, ["John", "x"])],

    # b. Apple & vegetables are food
    [("Food", True, ["Apple"])],
    [("Food", True, ["Vegetables"])],

    # c. Eats(x,y) ∧ ¬Killed(x) → Food(y)
    [("Eats", False, ["x","y"]), ("Killed", True, ["x"]), ("Food", True, ["y"])],

    # d. Anil eats peanuts; Anil alive
    [("Eats", True, ["Anil","Peanuts"])],
    [("Alive", True, ["Anil"])],

    # e. Harry eats everything Anil eats
    [("Eats", False, ["Anil","z"]), ("Eats", True, ["Harry","z"])],

    # f. Alive(x) → ¬Killed(x)
    [("Alive", False, ["x"]), ("Killed", False, ["x"])],

    # g. ¬Killed(x) → Alive(x)
    [("Killed", True, ["x"]), ("Alive", True, ["x"])]
]

# Query: h. John likes peanuts
query = [("Likes", False, ["John", "Peanuts"])]

def resolution(KB, query):
    clauses = KB + [query]  # add negated query
    print("\n--- Resolution Steps ---\n")

    while True:
        new = []
        for i in range(len(clauses)):
            for j in range(i+1, len(clauses)):
                resolvents = resolve(clauses[i], clauses[j])
                for r in resolvents:
                    print("Resolvent:", r)
                if [] in resolvents:
                    print("\nDerived empty clause → PROVED!\n")
                    return True
                new.extend(resolvents)

        if all(c in clauses for c in new):
            return False

        clauses.extend(new)


# ---------- Run ----------
result = resolution(KB, query)
print("Final Result:", result)



--- Resolution Steps ---

Resolvent: [('Likes', True, ['John', 'Apple'])]
Resolvent: [('Likes', True, ['John', 'Vegetables'])]
Resolvent: [('Likes', True, ['John', 'y']), ('Eats', False, ['y', 'y']), ('Killed', True, ['y'])]
Resolvent: [('Food', False, ['Peanuts'])]
Resolvent: [('Killed', True, ['Anil']), ('Food', True, ['Peanuts'])]
Resolvent: [('Killed', True, ['Harry']), ('Food', True, ['z']), ('Eats', False, ['Anil', 'z'])]
Resolvent: [('Eats', False, ['x', 'y']), ('Food', True, ['y']), ('Alive', False, ['x'])]
Resolvent: [('Eats', True, ['Harry', 'Peanuts'])]
Resolvent: [('Killed', False, ['Anil'])]
Resolvent: [('Killed', False, ['x']), ('Killed', True, ['x'])]
Resolvent: [('Alive', False, ['x']), ('Alive', True, ['x'])]
Resolvent: [('Likes', True, ['John', 'Apple'])]
Resolvent: [('Likes', True, ['John', 'Vegetables'])]
Resolvent: [('Likes', True, ['John', 'y']), ('Eats', False, ['y', 'y']), ('Killed', True, ['y'])]
Resolvent: [('Food', False, ['Peanuts'])]
Resolvent: [('Likes', 